Super, dass es jetzt funktioniert! Dein Projekt klingt spannend – eine Intranet-Seite für dein Heimnetzwerk, die zufällige Fotos von einer externen Festplatte anzeigt, auf einem Raspberry Pi 5. Da du mit Vite und vermutlich React arbeitest (aufgrund des Setups), werde ich dir einen Ansatz geben, wie du das umsetzen kannst. Wir müssen zwei Teile betrachten: den Frontend-Code (React) und eine Backend-Logik (z. B. mit Node.js), da der Zugriff auf eine externe Festplatte serverseitig erfolgen muss.

### Ansatz
1. **Backend**: Ein einfacher Node.js-Server liest die Fotos von der Festplatte und stellt eine API bereit.
2. **Frontend**: Deine React-App ruft die API an und zeigt ein zufälliges Foto an.
3. **Raspberry Pi**: Der Server läuft lokal auf dem Pi, und die externe Festplatte ist angeschlossen.

Ich gehe davon aus, dass deine Webseite bereits läuft und du nur das Feature für zufällige Fotos hinzufügen möchtest.

---

### Schritt 1: Backend (Node.js-Server)
Da Vite ein Frontend-Tool ist, brauchst du einen separaten Server, um auf die Festplatte zuzugreifen. Hier ist ein einfaches Node.js-Skript mit Express:

1. Erstelle einen neuen Ordner (z. B. `backend`) im Projektverzeichnis:
   ```bash
   mkdir backend
   cd backend
   npm init -y
   npm install express
   ```

2. Erstelle eine Datei `server.js` im `backend`-Ordner:
   ```javascript
   const express = require('express');
   const fs = require('fs').promises;
   const path = require('path');
   const app = express();
   const port = 5000; // Anderer Port als Vite (5173)

   // Pfad zur externen Festplatte (anpassen!)
   const photoDir = '/mnt/external/photos'; // Beispielpfad, siehe unten

   // API-Endpunkt für ein zufälliges Foto
   app.get('/api/random-photo', async (req, res) => {
     try {
       // Lies alle Dateien im Verzeichnis
       const files = await fs.readdir(photoDir);
       // Filtere nur Bilddateien (z. B. .jpg, .png)
       const photos = files.filter(file => /\.(jpg|jpeg|png|gif)$/i.test(file));
       if (photos.length === 0) {
         return res.status(404).send('Keine Fotos gefunden');
       }
       // Wähle ein zufälliges Foto
       const randomPhoto = photos[Math.floor(Math.random() * photos.length)];
       const photoPath = path.join(photoDir, randomPhoto);
       // Sende das Bild als Datei
       res.sendFile(photoPath);
     } catch (error) {
       console.error(error);
       res.status(500).send('Fehler beim Laden des Fotos');
     }
   });

   // Statische Dateien bereitstellen (optional, falls nötig)
   app.use('/photos', express.static(photoDir));

   app.listen(port, () => {
     console.log(`Server läuft auf http://localhost:${port}`);
   });
   ```

3. **Festplatte mounten**: Auf dem Raspberry Pi musst du die externe Festplatte einbinden (z. B. unter `/mnt/external`). Prüfe den Pfad mit:
   ```bash
   lsblk
   ```
   Mount-Befehl (z. B. für eine NTFS-Festplatte):
   ```bash
   sudo mkdir -p /mnt/external
   sudo mount /dev/sda1 /mnt/external
   ```
   Passe den Pfad in `server.js` (`photoDir`) entsprechend an.

4. Starte den Server:
   ```bash
   node server.js
   ```

---

### Schritt 2: Frontend (React mit Vite)
Jetzt passen wir deine React-App an, um das zufällige Foto anzuzeigen.

1. **Komponente erstellen**: Erstelle eine neue Datei, z. B. `RandomPhoto.jsx` im `src`-Ordner:
   ```jsx
   import { useState, useEffect } from 'react';

   function RandomPhoto() {
     const [photoUrl, setPhotoUrl] = useState('');

     // Funktion zum Abrufen eines zufälligen Fotos
     const fetchRandomPhoto = async () => {
       try {
         const response = await fetch('http://localhost:5000/api/random-photo');
         if (response.ok) {
           // Erstelle eine URL aus der Antwort
           const blob = await response.blob();
           const url = URL.createObjectURL(blob);
           setPhotoUrl(url);
         } else {
           console.error('Kein Foto gefunden');
         }
       } catch (error) {
         console.error('Fehler:', error);
       }
     };

     // Beim Laden ein Foto abrufen
     useEffect(() => {
       fetchRandomPhoto();
     }, []);

     return (
       <div>
         <h2>Zufälliges Foto</h2>
         {photoUrl ? (
           <img src={photoUrl} alt="Zufälliges Foto" style={{ maxWidth: '100%' }} />
         ) : (
           <p>Lade Foto...</p>
         )}
         <button onClick={fetchRandomPhoto}>Neues Foto</button>
       </div>
     );
   }

   export default RandomPhoto;
   ```

2. **Komponente einfügen**: Füge sie in deine Haupt-App ein, z. B. in `App.jsx`:
   ```jsx
   import RandomPhoto from './RandomPhoto';

   function App() {
     return (
       <div>
         <h1>Mein Heimnetzwerk-Intranet</h1>
         <RandomPhoto />
       </div>
     );
   }

   export default App;
   ```

3. **CORS-Probleme vermeiden**: Da Frontend (Port 5173) und Backend (Port 5000) unterschiedliche Ports nutzen, füge dem Backend CORS-Unterstützung hinzu:
   ```bash
   npm install cors
   ```
   In `server.js`:
   ```javascript
   const cors = require('cors');
   app.use(cors());
   ```

4. **Vite neu starten**: Stelle sicher, dass `npm run dev` läuft.

---

### Schritt 3: Auf dem Raspberry Pi ausführen
1. **Node.js auf dem Pi**: Stelle sicher, dass Node.js installiert ist:
   ```bash
   sudo apt update
   sudo apt install nodejs npm
   ```
2. Kopiere dein Projekt auf den Pi (z. B. via USB oder Git).
3. Starte Backend und Frontend:
   - Backend: `cd backend && node server.js`
   - Frontend: `cd dein-projekt && npm run dev`
4. Öffne `http://localhost:5173` im Browser (z. B. auf dem Pi oder einem anderen Gerät im Netzwerk).

---

### Ergebnis
- Die Seite zeigt ein zufälliges Foto von der Festplatte.
- Mit dem Button kannst du ein neues Foto laden.
- Alles läuft lokal auf deinem Raspberry Pi.

Falls du Fragen hast oder etwas nicht funktioniert, lass es mich wissen – z. B. Fehlermeldungen oder Probleme mit dem Festplattenpfad!